In [1]:
# Install uncommon packages (will supply requements.txt later)
#!pip install git+https://github.com/boudinfl/pke.git
#!pip install wordninja
#!pip install pyenchant

# For macos:
#brew install cmake
#git clone git@github.com:lloyd/yajl.git
#cd yajl
#./configure && make install


import pandas as pd
import numpy as np
import os
import json
import wordninja
from os import listdir
import re
import sys
import ast
import pke
import enchant
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import requests
import pickle
#import pycorpora
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
d = enchant.Dict("en_US") # ENGLISH Dictionary


# CHANGE THE PATH TO A DATASET HERE
basedir = os.path.join(os.sep, "Users", "artemiy", "Disk-O", "tyomkolton@mail.ru-mailru", "ML")


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/artemiy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/artemiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/artemiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organization_descriptions.csv"), index_col="uuid")
#data.iloc[:,1].to_csv('CB_Export_17_08_07/descr.txt')
print("loaded descriptions")
data1 = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organizations.csv"), low_memory=False, index_col="uuid")
print("loaded org infos")

loaded descriptions
loaded org infos


In [3]:
# Merge 2 csv files
# we can time to see which is more effective
#result = pd.concat([data, data1], axis=1, sort=False)
result1 = pd.merge(data, data1, how='inner', left_index=True, right_index=True)
print("merged")
# Delete all companies that do not have any descriptions
result1 = result1.dropna(subset=["description"])

merged


In [4]:
#print(((result.shape[0] - result1.shape[0])/result.shape[0]*100), "% of companies do not have any description")

In [7]:
result1.head()

,description,company_name,primary_role,permalink,domain,homepage_url,country_code,state_code,region,city,...,phone,facebook_url,linkedin_url,cb_url,logo_url,profile_image_url,twitter_url,alias,created_at,updated_at
uuid,,,,,,,,,,,,,,,,,,,,,
f5bb580f-d655-cf3f-9ade-eca5b3f2719f,"Android, Apple - iOS, Blackberry, Windows Phon...",VilarikA,company,/organization/vilarika,vilarika.com.br,http://vilarika.com.br/,BRA,NaN,Rio de Janeiro,Belo Horizonte,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/vilarika,https://www.crunchbase.com/organization/vilari...,http://public.crunchbase.com/t_api_images/v141...,NaN,NaN,2014-12-11 06:46:05,2016-09-07 00:03:51.67913
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,Formel D GmbH,company,/organization/formel-d-gmbh,formeld.com,http://www.formeld.com,DEU,NaN,DEU - Other,Troisdorf,...,+49 2241 9960,https://www.facebook.com/formeld,https://www.linkedin.com/company/formel-d-group,https://www.crunchbase.com/organization/formel...,https://www.crunchbase.com/organization/formel...,http://public.crunchbase.com/t_api_images/v148...,https://www.twitter.com/formeld_es,NaN,2016-06-01 06:58:37.692725,2017-07-18 07:22:19.15864
f5bd38d1-7719-2935-fb6c-defae39f5b93,[iNFoGooL](http://infogool.com) - The Informat...,infogool,company,/organization/infogool,infogool.com,http://infogool.com,NaN,NaN,NaN,NaN,...,8802393009,http://www.facebook.com/infogool,NaN,https://www.crunchbase.com/organization/infogool,https://www.crunchbase.com/organization/infogo...,http://public.crunchbase.com/t_api_images/v139...,https://www.twitter.com/infogool,NaN,2014-04-12 00:10:12,2016-09-08 22:02:05.585875
773adc18-132a-937d-8841-4f833e64dd56,The Jinfeng Gold Mine is an combined open pit ...,Jinfeng Mine,company,/organization/jinfeng-mine,NaN,NaN,CHN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/jinfen...,NaN,NaN,NaN,NaN,2016-12-07 06:43:06.955155,2016-12-07 06:45:09.14727
f5bdd48a-a09f-2f4c-bd71-4c4207c7731b,Peardoc offers online tools to convert HTML to...,Peardoc Solutions,company,/organization/peardoc-solutions,peardoc.com,http://www.peardoc.com,IND,NaN,Chennai,Chennai,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/peardo...,https://www.crunchbase.com/organization/peardo...,http://public.crunchbase.com/t_api_images/v140...,https://www.twitter.com/pear_doc,NaN,2014-07-17 10:06:03,2016-03-08 02:56:01.230586


In [8]:
len(result1)

353146

In [107]:
# Constuct a list of companies that have descriptions
#d_companies_list = list(result1.loc[:, "company_name"])
#d_companies_list = [company.lower() for company in d_companies_list]
#len(d_companies_list)

In [126]:
# Construct a list of companies from dataset folder
f_companies_list = os.listdir(os.path.join(basedir, "News_1"))
# Get rid of ".json" extension and apply uppercase
f_companies_list = [re.sub(".json", "", company) for company in f_companies_list]
print(len(f_companies_list))

20249


In [127]:
# Get rid of companies with names containing anything else than latin letters (as well as containing "_" separating multiple words in names)
f_companies_list = [company.lower() for company in f_companies_list if all(letter in string.ascii_letters for letter in company)]
print(len(f_companies_list))

# Get rid of companies whose full names consist of one single word from english dictionary
f_companies_list = [company for company in f_companies_list if not d.check(company.lower())]
print(len(f_companies_list))

12609
11385


In [110]:
def word_exists(word, apiKey=apiKey, apiUrl='http://api.wordnik.com/v4'):

    # Construct the API URL for a random-word query
    api_url = "{baseurl}/words.json/search/{word}".format(
        baseurl=apiUrl,
        word=word
    )

    parameters = {
        'api_key': apiKey,
    }

    # Perform the query and store the HTTP response object
    response = requests.get(api_url, params=parameters)

    # Convert the response content to a list
    # NOTE: The content is initially returned as a byte string
    word_object = json.loads(response.content)

    # Get the number word's instances in the dictionary
    if word_object:
        if word_object['searchResults'][0]['count'] > 0:
            result = True
        else:
            result = False
    else:
        result = False

    # Return the word as a string
    return result

# Same as the previous step (looking for english words) but with Wordnik API
apiUrl = 'http://api.wordnik.com/v4'
apiKey = 'b0a856e4cbe5c4e3f110d0527b901b7064f22a1c09e153547'
# Extract the list words from Wordnik that DO exist
wordnik_companies_list = [company for company in f_companies_list if word_exists(company)]
print(len(wordnik_companies_list))

367


In [104]:
# Tries to separate a name string into multiple words and if succeeds gets rid of the company (a lot of false positives)
# companies_list = [company for company in companies_list if len(wordninja.split(company.lower())) < 2]
# print(len(companies_list))

11385
11385
1


In [125]:
# These words DO exist in Wordnik dictionary
print((wordnik_companies_list))

['accend', 'acton', 'aerospike', 'afterschool', 'ais', 'alation', 'alector', 'alignable', 'amberjack', 'amicus', 'ananas', 'ansible', 'anyroad', 'aptera', 'aruspex', 'asana', 'attask', 'auris', 'authorly', 'aviso', 'ayr', 'backchat', 'backdoor', 'backtrace', 'bannerman', 'bastille', 'beme', 'bento', 'bevvy', 'biome', 'biscotti', 'bittorrent', 'blesh', 'bonafide', 'bonobos', 'bookbag', 'boombox', 'booyah', 'bottlenose', 'bowery', 'boxfish', 'brit', 'browster', 'carbonite', 'cardio', 'cartesian', 'cartogram', 'celly', 'centro', 'chargeback', 'chockstone', 'chronicity', 'clickable', 'cloze', 'containership', 'contentful', 'convo', 'copromote', 'courseload', 'courseloads', 'cringle', 'crossfader', 'cuculus', 'dekko', 'dextro', 'digby', 'doo', 'doozie', 'doppelganger', 'driverside', 'dropoff', 'dropship', 'duetto', 'eco', 'elastica', 'elsen', 'emma', 'endorphin', 'endplay', 'enow', 'entrada', 'epiphyte', 'eponym', 'eps', 'ess', 'everly', 'evolv', 'evolver', 'exogenesis', 'eyespot', 'fab', '

In [124]:
# Save for later use 
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open('wordnik_companies_list.txt', 'wb') as fp:
    pickle.dump(wordnik_companies_list, fp)
with open ('wordnik_companies_list.txt', 'rb') as fp:
    wordnik_companies_list = pickle.load(fp)

In [132]:
f_companies_list = [e for e in f_companies_list if e not in set(wordnik_companies_list)]
print(len(f_companies_list))
print(f_companies_list[0:100])

11018
['aaptiv', 'abaris', 'abbeypost', 'abcmob', 'abeo', 'abililife', 'abilto', 'abine', 'abiobot', 'ablexis', 'ablio', 'abodo', 'abogen', 'abom', 'aboutlife', 'aboutme', 'aboutone', 'aboutusorg', 'abra', 'absci', 'absmaterials', 'abusix', 'abvitro', 'abyrx', 'academiaedu', 'academixdirect', 'acadiasoft', 'accelera', 'accelereach', 'accelergy', 'accelgolf', 'accellos', 'accelo', 'accelops', 'acceptd', 'accera', 'accertify', 'accesssportsmediacom', 'acclaimd', 'acclarent', 'accountnow', 'accredible', 'accreon', 'accuitis', 'acculitx', 'accumen', 'accuradio', 'accuvein', 'accuwater', 'aceable', 'acebotai', 'acertiv', 'achaogen', 'achieveit', 'acompli', 'acopio', 'acousticeye', 'acquaintable', 'acquia', 'acrinta', 'acrisure', 'acrobatiq', 'acronis', 'acsian', 'actacell', 'actifio', 'actionality', 'actionsprout', 'actionx', 'activaero', 'actived', 'activegrid', 'activehours', 'activepath', 'activeprotective', 'activerain', 'activiter', 'activityhero', 'actmd', 'actualmeds', 'actuatedmedic

In [133]:
# Prepare the data frame of companies descriptions to be used in final data frame with articles
d_companies_df = result1
d_companies_df.loc[:,'company_name'] = d_companies_df.loc[:,'company_name'].str.lower()
#d_companies_df = d_companies_df[d_companies_df['company_name'].str.lower().isin(d_companies_list)]
#d_companies_df = d_companies_df[d_companies_df['company_name'].isin(d_companies_list)]
d_companies_df = d_companies_df.reset_index().set_index(keys='company_name')#, verify_integrity=True)



# Delete the companies which have the same name
d_companies_df = d_companies_df[~d_companies_df.index.duplicated(keep=False)]


d_companies_df = d_companies_df[['uuid', 'description']]
len(d_companies_df)

345890

In [134]:
# Delete companies outside or dataset of articles
companies_df = d_companies_df[d_companies_df.index.isin(f_companies_list)]
companies_list = companies_df.index.values
print(len(companies_df))

9658


In [ ]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open('companies_df.csv', 'wb') as fp:
    pickle.dump(companies_df, fp)

In [5]:
with open ('companies_df.csv', 'rb') as fp:
    companies_df = pickle.load(fp)
companies_df.head()
companies_list = companies_df.index.values
print(len(companies_df))

9658


In [19]:
### ONE WAY TO EXTRACT KEYWORDS ###

# With PKE Package
tmp = companies_df

# initialize keyphrase extraction model, here TopicRank
import pke



import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/artemiy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [22]:
counter = 0
for index, row in companies_df.iterrows():
    
    
    counter +=1
    #if counter >3:
    #    break
    
    extractor = pke.unsupervised.TopicRank()
    # load the content of the document, here document is expected to be in raw
    # format (i.e. a simple string of text) and preprocessing is carried out using nltk
    s = row["description"]
    #s = ' '.join(s.split())
    #s = "fwfewk w31231dmm! test test hello you piece of mind balls"
    extractor.read_text(s)

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives
    extractor.candidate_selection()

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    try:
        extractor.candidate_weighting()
        # N-best selection, keyphrases contains the 10 highest scored candidates as
        # (keyphrase, score) tuples
        keyphrases = extractor.get_n_best(n=10, stemming=False)
        keyphrases = sorted(keyphrases, key=lambda x: x[1], reverse=True)

        tmp.loc[index, 'kp_topicrank'] = str(keyphrases)
    
    except ValueError:
        print("Description for company " + index + 
              " (" + row['uuid'] + ")" +
              " is too short. Description: " + index)
        continue

        
        #if None in keyphrases:
        
    
tmp.tail()

Description for company ineomarketing (954b5316-3bcc-6081-d018-fec2c374d395) is too short. Description: ineomarketing
Description for company arrowsight (23bea55a-2e83-2b39-b53a-d8bd28d218e8) is too short. Description: arrowsight
Description for company looklive (6d3b1bed-302a-3331-6f43-87f91ca6a54b) is too short. Description: looklive
Description for company cityspark (f25f78b7-1a86-bf9d-e5da-24a538b81c61) is too short. Description: cityspark
Description for company rentabilities (aa8e1403-1db8-e5bf-6b5c-423bb826ccfd) is too short. Description: rentabilities
Description for company redkix (0510d73a-8709-bcae-5930-f047ee8b9db7) is too short. Description: redkix
Description for company shaser (538439c3-693e-0c54-76e3-f6c4c237370c) is too short. Description: shaser
Description for company bamboostr (83ccded4-e8ca-782f-2ef6-2c5b020fe48a) is too short. Description: bamboostr
Description for company metacert (83d3852a-f60d-2bac-24fe-afdbfb316fe9) is too short. Description: metacert
Descript

,uuid,description,kp_topicrank
company_name,,,
hooja,ff71401a-f5b0-1dd8-2b72-4a9e5d6ecba0,Hooja is a company focused on the mobile realm...,"[('service', 0.0785488995371759), ('hooja', 0...."
baseclick,d140620f-09bb-dd33-baaa-ecffff1d8bc6,baseclick GmbH is a start-up company funded an...,"[('technologies', 0.10249367027116733), ('star..."
biodetego,fef487a3-2505-e176-7bdc-d82db18525dc,"BioDetego is developing, VASPfore, a new bioma...","[('cancer treatment protocols', 0.105445975900..."
socialthreader,ffe12892-05c4-e8e4-d728-d9b54e7b3aeb,SocialThreader increases effectiveness of digi...,"[('campaign performance', 0.1546622347489285),..."
findally,cc693948-abf3-e3dc-f845-3e38ac718aab,Findally is an image-based product engine that...,"[('image-based product engine', 0.168320437934..."


In [ ]:
### Import the articles

In [26]:
# Get the list of paths to articles files and their sizes
paths = [(company, os.path.join(basedir, "News_1", "".join([company.upper(), ".json"]))) for company in companies_list]
paths = [path + (os.stat(path[1]).st_size,) for path in paths]
# Sort by sizes
paths = sorted(paths, key=lambda x: x[2], reverse=True)

full_size = 0
for f_size in paths:
    full_size = full_size + f_size[2]
print(full_size)

23680847067


In [ ]:
from jsonstreamer import JSONStreamer 
streamer = JSONStreamer()
for index_f, path in enumerate(paths):
    with open(path[1]) as file: 
        for line in file: 
            streamer.consume(line)

In [19]:
import ijson.backends.yajl2_cffi as ijson
def load_json(filename):
    with open(filename, 'rb') as fd:
        parser = ijson.parse(fd)
        return parser
def objects(self,file):
    key = '-'
    for prefix, event, value in ijson.parse(file):
        if prefix == '' and event == 'map_key':  # found new object at the root
            key = value  # mark the key value
            builder = ObjectBuilder()
        elif prefix.startswith(key):  # while at this key, build the object
            builder.event(event, value)
            if prefix == key+".item" and event == 'end_map':  # found the end of an object at the current key, yield
                yield key, builder.value 
for index_f, path in enumerate(paths):
    if index_f > 10:
        break
        
    print('MAMKA') 
    templist = load_json(path[1])
    with open(path[1], 'rb') as fd:
        #for key, value in objects(fd):
            #print(key, value)
        objectss = ijson.items(fd, "item")
        for item in objectss:
            gg = item
            
    #print(templist.read())

MAMKA
MAMKA
MAMKA
MAMKA
MAMKA
MAMKA
MAMKA
MAMKA
MAMKA
MAMKA
MAMKA


In [23]:
companies_df = tmp
len(k_company_df)

9936

In [24]:
k_company_df = k_company_df.dropna()
companies_list = k_company_df.index.get_values()
len(k_company_df)

9914

In [6]:
# Save for later use with keywords extraction and load
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
k_company_df.to_csv(os.path.join(basedir, 'temp_data', 'k_company_df.csv'), sep=";")

k_company_df = pd.read_csv(os.path.join(basedir, 'temp_data', 'k_company_df.csv'), sep=";")
k_company_df.head()

NameError: name 'k_company_df' is not defined

In [27]:
all_articles = pd.DataFrame()
num_articles = 0
regex = re.compile('[%s]' % re.escape(string.punctuation))


In [ ]:
for index_f, file in enumerate(paths):
    #if index_f > 0:
    #    break
    
    print(file)
    print(index_f)

    try:
        articles = pd.read_json(file[1])
    except ValueError:
        print("empty/faulty file")
        continue

    if "content" not in articles.columns:
        print("no content")
        continue

    articles.loc[:, "company"] = file[0]
    articles.loc[:, "uuid"] = k_company_df.loc[file[0].lower(), "uuid"]
    articles.loc[:, "description"] = k_company_df.loc[file[0].lower(), "description"]

    articles.dropna(subset=["content"], inplace=True)

    texts = [regex.sub('', i).lower() for i in articles.loc[:, "content"]]
    keyphrases = ast.literal_eval(k_company_df.loc[file[0].lower(), "kp_topicrank"])
    num_articles = num_articles + len(texts)

    # Check whether the any of the keywords exist in the text
    bi = [1 
             if any([keyphrase in text 
                     for (keyphrase, prob) in (keyphrases)]) and 250 < len(str.split(text)) < 500 
             else 0
            for text in texts]

    articles.loc[:, "kp_topicrank_bi"] = bi
    articles.dropna(subset=["content"], inplace=True)

    all_articles = all_articles.append(articles, ignore_index=True, sort=False)
    
    

In [59]:
all_articles["kp_topicrank_bi"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
       ... 
5662    0.0
5663    0.0
5664    0.0
5665    1.0
5666    1.0
5667    0.0
5668    0.0
5669    0.0
5670    0.0
5671    0.0
5672    0.0
5673    0.0
5674    0.0
5675    0.0
5676    0.0
5677    0.0
5678    0.0
5679    0.0
5680    0.0
5681    0.0
5682    0.0
5683    0.0
5684    0.0
5685    0.0
5686    0.0
5687    0.0
5688    0.0
5689    1.0
5690    1.0
5691    0.0
Name: kp_topicrank_bi, Length: 5692, dtype: float64

In [148]:
print(str(len(all_articles/num_articles)) + "% of articles were are containing corresponding keyphrases extracted from the descriptions of the companies")

0% of articles were are containing corresponding keyphrases extracted from the descriptions of the companies


In [ ]:
# Save dataset
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
all_articles.to_csv(os.path.join(basedir, 'temp_data', 'all_articles.csv'), sep=";")
all_articles.head()

In [20]:
print(companies_list)

['ABRA', 'ACRONIS', 'ACTON', 'AIRSTONE', 'ALBERT', 'ALEA', 'ALIANZA', 'AMARA', 'ANDO', 'ARKIN', 'ARTSPACE', 'ASANA', 'ASOKA', 'ASTRID', 'ATHOS', 'AVA', 'AYR', 'BACKDOOR', 'BANNERMAN', 'BARKLY', 'BASTILLE', 'BBB', 'BEBO', 'BENTO', 'BIA', 'BINDO', 'BIOME', 'BIRDBOX', 'BITTORRENT', 'BIX', 'BIZEN', 'BOARDVOTE', 'BONOBOS', 'BOTTLENOSE', 'BOWERY', 'BRAINTREE', 'BRIT', 'BUDDYTV', 'CAIS', 'CALERA', 'CALVIN', 'CARTESIAN', 'CASPER', 'CASSATT', 'CENTRO', 'CERBERUS', 'CHAI', 'CLARK', 'CLEO', 'CLICKABLE', 'CLOE', 'COTOPAXI', 'CRONO', 'CROSSFADER', 'DAILYMOTION', 'DELOS', 'DIGBY', 'DIY', 'DOO', 'DOPPELGANGER', 'ECO', 'ELASTICA', 'ELEMENTUM', 'ELLIE', 'ELɘ', 'EMMA', 'ENDORPHIN', 'EPIPHYTE', 'EPS', 'ESS', 'EVERLY', 'EXCALIBUR', 'EXOGENESIS', 'EYESPOT', 'FAB', 'FACEBOOK', 'FANBASE', 'FARADAY', 'FIDELIS', 'FLASHPOINT', 'FLAVOUR', 'FLYER', 'FLYNN', 'FONDU', 'FORMSPRING', 'FRONTO', 'FRS', 'FULHAM', 'FUZE', 'FY', 'GAMETIME', 'GAMEZONE', 'GAMGEE', 'GELI', 'GENI', 'GOLDSTAR', 'GOLGI', 'GORGIAS', 'GRO', 'HALL

In [56]:
len(articles)

76920

In [175]:
all_articles.tail()

,UNPARSED,autor,byline,contact,content,dateline,distribution,error,error_code,grafic,...,pub_type,rubrik,section,series,source,company,uuid,description,country,correction_date
4395,"AFX - Asia\n\n November 2,...",NaN,NaN,NaN,"AFX - Asia\n\n November 2,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4396,"AFX - Asia\n\n August 16,...",NaN,NaN,NaN,"AFX - Asia\n\n August 16,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4397,Manila Times\n\n J...,NaN,NaN,NaN,Manila Times\n\n J...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4398,"AFX - Asia\n\n May 24, 2...",NaN,NaN,NaN,"AFX - Asia\n\n May 24, 2...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4399,"AFX - Asia\n\n April 22,...",NaN,NaN,NaN,"AFX - Asia\n\n April 22,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
